In [3]:
import sys
import os
import requests
import json
import csv
import decimal
import copy

#sys.path.append("/smartcode/util")
import util
from importlib import reload
reload(util)

<module 'util' from '/smartcode/setup/challenges/util/__init__.py'>

In [2]:
!pwd

/smartcode/setup/challenges


# Challenge setup geth

## Start environment: smartenv-geth & smartenv-web3py 

We will run the server/geth node as well as the smar>tenv-web3py client docker containers on the same host machine. 

TODO



## Connect and check chain state

Note that for *PoA* nodes the middleware needs to be added.
Therefore, set the `poa` flag to true when connecting. 

In [31]:
SRVIP = "172.18.0.8"
SRVPORT = "8545"
SRVNETVER = "1337"

#w3 = web3.Web3(web3.Web3.HTTPProvider("http://172.18.0.1:8545"))
#w3=util.connect(host="172.18.0.5",port="8544",poa=True) 
w3=util.connect(host=SRVIP,port=SRVPORT,poa=True)
# check if connection was successful
assert w3.isConnected()

In [10]:
w3.api #web3 version used

'5.20.0'

In [11]:
w3.clientVersion # remote client version to which w3 is connected 

'Geth/bob/v1.10.4-unstable-0e9c7d56-20210607/linux-amd64/go1.15.4'

In [12]:
assert w3.net.version == SRVNETVER
w3.net.version

'1337'

In [13]:
w3.eth.blockNumber

33004

In [14]:
w3.eth.getBlock(w3.eth.blockNumber)["gasLimit"]

8000000

In [15]:
w3.eth.mining

False

In [16]:
w3.eth.syncing

False

### Networ check

In [17]:
w3.net.peerCount

1

In [18]:
w3.geth.admin.peers()

[AttributeDict({'enode': 'enode://c59829866405d6aa173390d3de8f6623093cbd5ea86d7c3ea67296059a084b7c02711dc7f4d5bb0d5efec08fc79461c849320fe3116cfa17a330c1fd6a482416@131.130.126.71:30303',
  'id': '6362800dc7dda26b4125c12dbe29e780b6a6b7f4b7bc3dfbd30e3da9aa8c2fb3',
  'name': 'Geth/alice/v1.10.3-stable-991384a7/linux-amd64/go1.15.4',
  'caps': ['eth/65', 'eth/66', 'snap/1'],
  'network': AttributeDict({'localAddress': '172.18.0.8:37798',
   'remoteAddress': '131.130.126.71:30303',
   'inbound': False,
   'trusted': False,
   'static': False}),
  'protocols': AttributeDict({'eth': AttributeDict({'version': 66,
    'difficulty': 66007,
    'head': '0xd937660b597f03a8a33214ad236627ebb674572e09b713ce105dce6528740113'}),
   'snap': AttributeDict({'version': 1})})})]

In [19]:
w3.net.listening

True

In [20]:
w3.eth.accounts

['0xa1273F73C607Bd0af4D2916f4C9e6A550581dCA6',
 '0x01A8719E91D8c0Fbecb28d41DA5F64e6427A7d94']

### Export (correct) enode info for server (docker container)

In [23]:
!cd /smartcode && mkdir -p ./generated_config

In [36]:
poa_node_con = False
enode_file = "./generated_config/enodes"
# only needed if running directly on PoA node server:
#invalid_ip_port = "172.18.0.5:30303"
invalid_ip_port = "127.0.0.1:30303"
valid_ip_port = "128.131.169.35:30303"

if poa_node_con:
    if w3.clientVersion.startswith('Parity'):
        enode = w3.parity.enode()
    elif w3.clientVersion.startswith('Geth'):
        enode = w3.geth.admin.nodeInfo()['enode']
    enode = enode.replace(invalid_ip_port,valid_ip_port)
else:
    enode = w3.geth.admin.peers()[0]["enode"]
enode

'enode://c59829866405d6aa173390d3de8f6623093cbd5ea86d7c3ea67296059a084b7c02711dc7f4d5bb0d5efec08fc79461c849320fe3116cfa17a330c1fd6a482416@131.130.126.71:30303'

In [ ]:
# Write enode to file:
!cd /smartcode && echo $enode > $enode_file

In [34]:
# block 0 ist the genesis block
genesis_block = w3.eth.getBlock(0)
genesis_block

AttributeDict({'difficulty': 1,
 'proofOfAuthorityData': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000e63419b7c0be62a6127923fb70c32a5ca1926a160000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'gasLimit': 15000000,
 'gasUsed': 0,
 'hash': HexBytes('0xdede33d41529ddf39473ee748e66921e107aa6431db6fc2481dbcef833c583e4'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'miner': '0x00000000000

## Import students

In [40]:
!ls -la /smartcode/participants/participants.csv

-rw-rw-r-- 1 smartcode smartcode 934 Nov 17 18:34 /smartcode/participants/participants.csv


In [57]:
in_file = "/smartcode/participants/participants.csv"

In [58]:
!head -n2 $in_file

Aljosha,Judmayer,aljosha.judmayer@univie.ac.at,1
Barbara,Limbeck-Lilienau,barbara.limbeck-lilienau@univie.ac.at,0


In [59]:
!file $in_file # get file format/encoding

/smartcode/participants/participants.csv: CSV text


In [60]:
!wc -l $in_file # check number of stutend and compare later 

18 /smartcode/participants/participants.csv


In [61]:
students_list = list()
students_count = 0

in_fieldnames = ['firstname', 'lastname','email','matr']

In [62]:
def read_students(path):
    global students_count
    global students_list
    with open(path,'r',encoding='utf-8') as f_input:
    #with open(path,'r',encoding='utf-16') as f_input:
    #with open(path,'r',encoding='latin-1') as f_mail:
        #sreader = csv.DictReader(f_input, fieldnames=in_fieldnames, delimiter='\t',quotechar='\"')
        sreader = csv.DictReader(f_input, fieldnames=in_fieldnames, delimiter=',',quotechar='\"')
        for i,row in enumerate(sreader):
            #print(i,row)
            student_dict = dict()
            for field in in_fieldnames:
                    student_dict[field]=row[field]
            students_list.append(student_dict)
            #print(students_list)
    students_count = i+1
    return students_count

In [63]:
read_students(in_file)

18

In [64]:
students_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'aljosha.judmayer@univie.ac.at',
 'matr': '1'}

In [65]:
students_list[-1]

{'firstname': 'Nicholas',
 'lastname': 'Stifter',
 'email': 'nicholass83@univie.ac.at',
 'matr': '2'}

In [67]:
nr_students = len(students_list)
nr_students

18

## Create keys

In [68]:
!cat /smartcode/create_accounts.sh 

#!/bin/bash


if [ -z "${1}" ] && [ -z "${PARTICIPANTS+x}" ];
then
    echo "PARTICIPANTS = No number given e.g., '42'"
    exit 2
elif [ ! -z "${1}" ];
then
    export PARTICIPANTS="${1}"
fi

export DATADIR="./datadir/accountmgt"

echo "DATADIR = ${DATADIR}"
echo "PARTICIPANTS = ${PARTICIPANTS}"

mkdir -p "./generated_config/keystore/"

#for i in {0..90}};
for i in $(seq 1 ${PARTICIPANTS});
do 
	echo $i
	bash ./geth_account.sh new
done

cp -r ./datadir/accountmgt/keystore/* ./generated_config/keystore/


In [69]:
!cat /smartcode/geth_account.sh

#!/bin/bash
#
#
# Run with docker:
# $ PWFILE=./passwordfile DATADIR=./datadir/accountmgt/ bash geth_account.sh new

if [ -z "${PWFILE+x}" ];
then
	echo "PWFILE environment variable missing"
	echo "No Password file given using default './passwordfile'"
	export PWFILE='./passwordfile'
fi
echo "PWFILE = ${PWFILE}"

if [ -f "${PWFILE}" ];
then
	echo "using stored password ... "	
else
	pwgen -s -B 32 1 > ${PWFILE}
fi

if [ -z "${DATADIR+x}" ];
then
        echo "DATADIR environment variable missing"
	echo "No Datadir given e.g., '/datadir/alice/'"
	exit 2
fi
echo "DATADIR = ${DATADIR}"

# run container without network
# just to generate and manage keys
docker run \
	--mount type=bind,source=$(pwd),target=/smartenv \
	-it smartenv:latest \
		geth \
		--datadir "${DATADIR}" \
		--nodiscover \
		--maxpeers 0 \
		--ipcdisable \
		--verbosity 6  \
		--password "${PWFILE}" \
		account "$1" "$2" "$3" 



## Assign addresses to users (testaccounts & students)

In [70]:
#### CHANGE ####
 
# path where the raw ethereuem keys are located:
#source_path="./test/keystore/" 
source_path="/smartcode/generated_config/keystore"

# Either run with test users or with all students:
#users_list = copy.deepcopy(testers_list)
users_list = copy.deepcopy(students_list)

#users_list = copy.deepcopy(testers_list)
#users_list.extend(copy.deepcopy(students_list)

#### CHANGE END ####

In [71]:
# print some user as test before assignment
print(len(users_list))
users_list[0]

18


{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'aljosha.judmayer@univie.ac.at',
 'matr': '1'}

In [72]:
!ls /smartcode/generated_config

enodes	keystore


In [73]:
!cd $source_path && pwd && ls | head -n1

/smartcode/generated_config/keystore
UTC--2020-11-17T17-45-00.970135674Z--0f60ea75ffa7ebdb5ac2924056d3dcad33504932


In [74]:
!cd $source_path && ls ./ | head -n1 | xargs -Irpl cat rpl

{"address":"0f60ea75ffa7ebdb5ac2924056d3dcad33504932","crypto":{"cipher":"aes-128-ctr","ciphertext":"60ace180957c04e2a17379d10a0bdd784ee3ce7eb19e7701056c01945857d1aa","cipherparams":{"iv":"4b9219cab3e20b0f25f54115d4da4f35"},"kdf":"scrypt","kdfparams":{"dklen":32,"n":262144,"p":1,"r":8,"salt":"41b6f3ad105b3f3d0aed80dd86c8e17ecd57b9218c4708dfc2ce02f2ed4ee2ad"},"mac":"663d8bb101c70fb9a7e6ab56b9603e225d69b8aa19e1d6940557efccb4cffa90"},"id":"4c09cecd-892d-4c06-92b6-a5fce96ff614","version":3}

In [75]:
def read_keypairs(path):
    """read keypairs from keystore
    """
    _keypairs = list()
    # read out keystore folder 
    for root,dirs,files in os.walk(path):
        #print("root = {}".format(root))
        #print("dirs = {}".format(dirs))
        for f in files:
            with open(root + "/" + f,"r",encoding="utf-8") as f_input:
                data = json.loads(f_input.read()) # load all key pair data from file
                data["filename"] = f # add file name to data for later
                _keypairs.append(data)
    return _keypairs   

In [76]:
def assign_addresses(keystore_path,users,attach_path="/smartcode/generated_config/attach"):
    """ map keypairs to users and add info to user dict 
    """
    keypairs = read_keypairs(keystore_path)
    assert len(keypairs) >= len(users)
    
    for u in users:
        kp = keypairs.pop()
        u["account"] = kp["address"]
        #u["keypair"] = kp
        #u["key"] = kp["crypto"]["ciphertext"]
        u["filename"] = kp["filename"]
        
        arg_dest = attach_path + "/" + u["account"]
        !mkdir -p $arg_dest
        arg_src = keystore_path + "/" + kp["filename"]
        !cp $arg_src $arg_dest
        u["attach"] = "/tmp/attach/" + u["account"] + ".tar.gz"
        print(".",end=".")
    return users

In [77]:
attach_path="/smartcode/generated_config/attach"
#!rm -rf attach_path
assign_addresses(source_path,users_list,attach_path)

# print some user as test after assignment
users_list[0]

....................................

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'aljosha.judmayer@univie.ac.at',
 'matr': '1',
 'account': '6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'filename': 'UTC--2020-11-17T17-45-22.602842072Z--6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'attach': '/tmp/attach/6d031a36dd43f2fc2539037f0bf8a0c818ef9b59.tar.gz'}

In [78]:
def getBalance(address):
    address = w3.toChecksumAddress(address)
    return w3.fromWei(w3.eth.getBalance(address),'ether')

In [79]:
getBalance(users_list[0]["account"])

0

In [80]:
w3.eth.accounts

['0xe63419B7c0be62A6127923fb70C32A5ca1926a16']

In [81]:
for a in w3.eth.accounts:
    print(a,": ",getBalance(a))

0xe63419B7c0be62A6127923fb70C32A5ca1926a16 :  100000000


In [82]:
w3.eth.defaultAccount = w3.eth.accounts[0]

In [83]:
w3.eth.defaultAccount

'0xe63419B7c0be62A6127923fb70C32A5ca1926a16'

In [84]:
def send_accounts_money(users,amount):
    for u in users:
        account = w3.toChecksumAddress(u["account"])
        u["seedbalance"] = amount
        w3.eth.sendTransaction({"from":w3.eth.defaultAccount,"to":account,"value":amount})
    return

In [85]:
send_accounts_money(users_list,16*10**18)

In [88]:
getBalance(users_list[0]["account"])

Decimal('16')

In [89]:
getBalance(users_list[1]["account"])

Decimal('16')

In [90]:
def check_accounts_money(users,predicate=(lambda a, b: a == b)):
    for u in users:
        u_balance = w3.eth.getBalance(w3.toChecksumAddress(u["account"]))
        u["balance"] = u_balance
        u_seedbalance = u["seedbalance"]
        if predicate(u_balance,u_seedbalance) == False:
            print("Predicate violation for:\n",u)

In [91]:
check_accounts_money(users_list)

In [92]:
check_accounts_money(users_list,predicate=(lambda a, b: a >= b))
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'aljosha.judmayer@univie.ac.at',
 'matr': '1',
 'account': '6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'filename': 'UTC--2020-11-17T17-45-22.602842072Z--6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'attach': '/tmp/attach/6d031a36dd43f2fc2539037f0bf8a0c818ef9b59.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000}

### Test manual account handling

In [93]:
for a in w3.eth.accounts:
    print(a,": ",getBalance(a))

0xe63419B7c0be62A6127923fb70C32A5ca1926a16 :  99999712


In [94]:
w3.eth.sendTransaction({'from':w3.eth.accounts[0], 
                        'to':"0xEe07C32520Ed77679407ad373A6F10667B5498E2", 
                        'value':10**18})

HexBytes('0xf0c7537d81b2ad44577a12e81d5d9e7b38a7e710e86d8b67bb79f0755124ba03')

### Deployment functions

In [95]:
sys.path.append("/smartcode/util")
import util
from importlib import reload
reload(util)

<module 'util' from '/smartcode/notebooks/challenges/util/__init__.py'>

In [96]:
def forall_compile_and_deploy(users,path,contract_name,value,force_argument=None,argument2=None):
    print("Started for {} users:".format(len(users)))
    for u in users:
        if force_argument is not None:
            print("*",end="")
            #print("Force argument: {}".format(force_argument))
            u_addr = force_argument
        else:
            u_addr = w3.toChecksumAddress(u["account"])
            
        if argument2 is not None and argument2 is True:
            print("~",end="")
            #print("Double sha3 argument2: {}".format(argument2))
            tx_hash = util.compile_and_deploy_contract(path,
                                                   patch_api=False,
                                                   concise=True,
                                                   argument=u_addr,
                                                   argument2=w3.toInt(hexstr=w3.sha3(text=w3.sha3(text=u_addr).hex()).hex()),
                                                   wait=False,
                                                   value=value)
        elif argument2 is not None and isinstance(argument2,type(str())):
            print("+",end="")
            #print("Custom argument2: {}".format(argument2))
            tx_hash = util.compile_and_deploy_contract(path,
                                                   patch_api=False,
                                                   concise=True,
                                                   argument=u_addr,
                                                   argument2=argument2,
                                                   wait=False,
                                                   value=value)
        else:
            print(".",end="")
            tx_hash = util.compile_and_deploy_contract(path,
                                                   patch_api=False,
                                                   concise=True,
                                                   argument=u_addr,
                                                   wait=False,
                                                   value=value)
        # returns HexBytes
        # import hexbytes
        # help(hexbytes.main)
        u[contract_name + "_tx"] = tx_hash.hex()
        u[contract_name + "_seedvalue"] = value
        
    for u in users:
        tx_receipt = w3.eth.waitForTransactionReceipt(u[contract_name + "_tx"])
        u[contract_name + "_addr"] = tx_receipt['contractAddress']
        print("-",end="")
        
    return users

In [97]:
def forall_check_contract(users,c_path,c_name,predicate=(lambda a, b: a == b)):
    # Check for all users if the contract is initialized to the respective user
    # and that the balance is the seed value
    print("Started for {} users:".format(len(users)))
    for u in users:
        c_abi = util.compile_contract_with_libs(compiler_path="solc",src_path=c_path)["abi"]
        u_c_addr = u[c_name + "_addr"]
        u_c_instance = util.get_contract_instance(u_c_addr,c_abi,concise=False)
    
        # check owner
        #print(u_c_instance.functions.getStudent().call())
        #print(w3.toChecksumAddress(u["account"]))
        assert u_c_instance.functions.getStudent().call() == w3.toChecksumAddress(u["account"])   
        # check balance
        u_c_seedvalue = u[c_name + "_seedvalue"]
        u_c_balance = w3.eth.getBalance(u_c_addr)   
        u[c_name + "_balance"] = u_c_balance
        if predicate(u_c_balance,u_c_seedvalue) == False:
            print("Predicate violation for:\n",u)
        print(".",end="")
    return users

In [98]:
def copy_challenge_metadata(users,challenge_path,challenge_name,attach_path="./attach"):
    # copy challenge metadata to individual user folders helper function
    for d in os.listdir(attach_path):
        #print(d)
        if os.path.isdir(os.path.join(attach_path, d)):
            #print(d)
            arg_dest_dir = attach_path + "/" + d + "/" + challenge_name
            print(arg_dest_dir)
            !mkdir -p $arg_dest_dir
            !cp -r $challenge_path $arg_dest_dir

### Challange: notawallet

In [99]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'aljosha.judmayer@univie.ac.at',
 'matr': '1',
 'account': '6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'filename': 'UTC--2020-11-17T17-45-22.602842072Z--6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'attach': '/tmp/attach/6d031a36dd43f2fc2539037f0bf8a0c818ef9b59.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000}

In [100]:
users_list[-1]

{'firstname': 'Nicholas',
 'lastname': 'Stifter',
 'email': 'nicholass83@univie.ac.at',
 'matr': '2',
 'account': '16fbacd7ed127eb7ee74118992c0010160ac5ad0',
 'filename': 'UTC--2020-11-17T17-45-49.937324003Z--16fbacd7ed127eb7ee74118992c0010160ac5ad0',
 'attach': '/tmp/attach/16fbacd7ed127eb7ee74118992c0010160ac5ad0.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000}

In [101]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.5.4+commit.9549d8ff.Linux.g++


In [102]:
!solc --overwrite -o "/smartcode/challenges/notawallet/publish/" --abi --bin "/smartcode/challenges/notawallet/publish/NotAWallet.sol"

Compiler run successful. Artifact(s) can be found in directory /smartcode/challenges/notawallet/publish/.


In [105]:
## Connections refresh maybe needed
w3=util.connect(host="172.18.0.5",port="8544",poa=True) 
assert w3.isConnected()

In [106]:
path="/smartcode/challenges/notawallet/publish/NotAWallet.sol"
challange_name="notawallet"
seedvalue=10*10**18

forall_compile_and_deploy(users_list,path,challange_name,seedvalue)
users_list[0]

Started for 18 users:
..................------------------

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'aljosha.judmayer@univie.ac.at',
 'matr': '1',
 'account': '6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'filename': 'UTC--2020-11-17T17-45-22.602842072Z--6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'attach': '/tmp/attach/6d031a36dd43f2fc2539037f0bf8a0c818ef9b59.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x4e09544c87b640268c02286e161a8001e754471b6c1c240882df20717e9fb3fd',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x2e52F50386de9fDF7177cb7C886B525595AAb634'}

In [108]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'aljosha.judmayer@univie.ac.at',
 'matr': '1',
 'account': '6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'filename': 'UTC--2020-11-17T17-45-22.602842072Z--6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'attach': '/tmp/attach/6d031a36dd43f2fc2539037f0bf8a0c818ef9b59.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x4e09544c87b640268c02286e161a8001e754471b6c1c240882df20717e9fb3fd',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x2e52F50386de9fDF7177cb7C886B525595AAb634'}

In [109]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.5.4+commit.9549d8ff.Linux.g++


In [110]:
c_abi = util.compile_contract_with_libs(compiler_path="solc",src_path="/smartcode/challenges/notawallet/publish/NotAWallet.sol")["abi"]
c_abi

'[{"constant":false,"inputs":[{"name":"oldowner","type":"address"}],"name":"removeOwner","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"testowner","type":"address"}],"name":"isOwner","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"student","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"newowner","type":"address"}],"name":"addOwner","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"owner","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name"

In [111]:
notawallet_addr = users_list[0]["notawallet_addr"]
notawallet_addr

'0x2e52F50386de9fDF7177cb7C886B525595AAb634'

In [112]:
u_c_instance = util.get_contract_instance(notawallet_addr,c_abi,concise=False)

In [113]:
assert u_c_instance.functions.getStudent().call() == w3.toChecksumAddress(users_list[0]["account"])
u_c_instance.functions.getStudent().call()

'0x6d031A36DD43f2fC2539037f0Bf8A0C818Ef9B59'

In [114]:
path="/smartcode/challenges/notawallet/publish/NotAWallet.sol"
challange_name="notawallet"
forall_check_contract(users_list,path,challange_name)
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a < b))
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a == 10**18))
users_list[0]

Started for 18 users:
..................

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'aljosha.judmayer@univie.ac.at',
 'matr': '1',
 'account': '6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'filename': 'UTC--2020-11-17T17-45-22.602842072Z--6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'attach': '/tmp/attach/6d031a36dd43f2fc2539037f0bf8a0c818ef9b59.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x4e09544c87b640268c02286e161a8001e754471b6c1c240882df20717e9fb3fd',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x2e52F50386de9fDF7177cb7C886B525595AAb634',
 'notawallet_balance': 10000000000000000000}

In [115]:
# copy challenge metadata
challenge_path="/smartcode/challenges/notawallet/publish/*"
attach_path="/smartcode/generated_config/attach"
challenge_name="notawallet"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartcode/generated_config/attach/d7bdb839a8b3500ff9db3a536de3fdc1ef90a7c6/notawallet
/smartcode/generated_config/attach/9f2874ebca0e202e6f32deec20e7c03b1b0b5d8c/notawallet
/smartcode/generated_config/attach/561f30aa118eb27a1e717a80e011e3ce92ad1e39/notawallet
/smartcode/generated_config/attach/3e93bc40c509e3b8f8eb4a0e02bfdfcb7dcb16cd/notawallet
/smartcode/generated_config/attach/55941e806f8ff9a40db959e08f2fb9b466141667/notawallet
/smartcode/generated_config/attach/0f60ea75ffa7ebdb5ac2924056d3dcad33504932/notawallet
/smartcode/generated_config/attach/e2f6f489afefd074b4907977bd125f6bb20d209d/notawallet
/smartcode/generated_config/attach/7f44a60a059be711cfa2f8b5809019813a0ce03d/notawallet
/smartcode/generated_config/attach/e87f9b2bb30497579c98fb70a6f21275ad18a4b2/notawallet
/smartcode/generated_config/attach/2613c370f5e2646d2fba8a7652dca4544ee3bfa5/notawallet
/smartcode/generated_config/attach/beab9297f2b63e43541b918c75ead656d231b986/notawallet
/smartcode/generated_config/attach/c43463f7

## Challenge: badparity

In [116]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'aljosha.judmayer@univie.ac.at',
 'matr': '1',
 'account': '6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'filename': 'UTC--2020-11-17T17-45-22.602842072Z--6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'attach': '/tmp/attach/6d031a36dd43f2fc2539037f0bf8a0c818ef9b59.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x4e09544c87b640268c02286e161a8001e754471b6c1c240882df20717e9fb3fd',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x2e52F50386de9fDF7177cb7C886B525595AAb634',
 'notawallet_balance': 10000000000000000000}

In [117]:
!cat "/smartcode/challenges/badparity/publish/Wallet.sol"

pragma solidity ^0.5.4;

contract Wallet {
    address walletLibrary;
    address payable owner;
    address payable student;

    event LogValue(uint256 exitcode,uint256 amount);

    // constructor, called once when this contract is created 
    constructor(address payable _student, address lib) public payable {
        student = _student;  
        walletLibrary = lib; // hardcode lib address at deploy time
        // init the owner with the respective lib contract
        walletLibrary.delegatecall(abi.encodeWithSignature("initWallet(address)", msg.sender));
    }

    function getOwner() public view returns (address payable) {
        return owner;
    } 

    function getWalletLibrary() public view returns (address) {
        return walletLibrary;
    }

    function withdraw(uint256 amount) public returns (bool) {
        (bool success, bytes memory data) = walletLibrary.delegatecall(abi.encodeWithSignature("withdraw(uint256)", amount));
        if ( success ) {
            emit

In [118]:
!solc --overwrite -o "/smartcode/challenges/badparity/publish/" --abi --bin "/smartcode/challenges/badparity/publish/Wallet.sol"

/smartcode/challenges/badparity/publish/Wallet.sol:15:9: Warning: Return value of low-level calls not used.
        walletLibrary.delegatecall(abi.encodeWithSignature("initWallet(address)", msg.sender));
        ^------------------------------------------------------------------------------------^
/smartcode/challenges/badparity/publish/Wallet.sol:49:9: Warning: Return value of low-level calls not used.
        walletLibrary.delegatecall(msg.data);
        ^----------------------------------^
/smartcode/challenges/badparity/publish/Wallet.sol:27:24: Warning: Unused local variable.
        (bool success, bytes memory data) = walletLibrary.delegatecall(abi.encodeWithSignature("withdraw(uint256)", amount));
                       ^---------------^
/smartcode/challenges/badparity/publish/Wallet.sol:37:24: Warning: Unused local variable.
        (bool success, bytes memory data) = walletLibrary.delegatecall(abi.encodeWithSignature("changeOwner(address)", new_owner));
                       

In [119]:
!solc --overwrite -o "/smartcode/challenges/badparity/publish/" --abi --bin "/smartcode/challenges/badparity/publish/WalletLibrary.sol"

Compiler run successful. Artifact(s) can be found in directory /smartcode/challenges/badparity/publish/.


In [120]:
# compile and deploy lib once for all users
path="/smartcode/challenges/badparity/publish/WalletLibrary.sol"

lib = util.compile_and_deploy_contract(path,
                                 patch_api=False,
                                 concise=False)
lib.address

'0x5bE9a46AEDE5cdF200E7aCfBfd6FcA7cF336F9CE'

In [121]:
path="/smartcode/challenges/badparity/publish/Wallet.sol"
challange_name="badparity"
seedvalue=30*10**18

forall_compile_and_deploy(users_list,path,challange_name,seedvalue,argument2=lib.address)

for u in users_list:
    u[challange_name + "_lib"] = lib.address

users_list[0]

Started for 18 users:
++++++++++++++++++------------------

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'aljosha.judmayer@univie.ac.at',
 'matr': '1',
 'account': '6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'filename': 'UTC--2020-11-17T17-45-22.602842072Z--6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'attach': '/tmp/attach/6d031a36dd43f2fc2539037f0bf8a0c818ef9b59.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x4e09544c87b640268c02286e161a8001e754471b6c1c240882df20717e9fb3fd',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x2e52F50386de9fDF7177cb7C886B525595AAb634',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x952943aab82306d562752dc63672eb3d52fa3908ede0c9cb6830a395570a905e',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0x96cdA09224CE798b9d39742801A34486E1599a3f',
 'badparity_lib': '0x5bE9a46AEDE5cdF200E7aCfBfd6FcA7cF336F9CE'}

In [122]:
# spot check of one contract
c_abi = util.compile_contract_with_libs("solc",path)["abi"]
u_c_addr = users_list[0][challange_name + "_addr"]
u_c_instance = util.get_contract_instance(u_c_addr,c_abi,concise=True)

assert users_list[0][challange_name + "_lib"] == lib.address
    
# check lib
assert str(u_c_instance.getWalletLibrary()).lower() == str(lib.address).lower()
assert str(u_c_instance.getOwner()).lower() == str(w3.eth.accounts[0]).lower()

# check balance
assert w3.eth.getBalance(u_c_addr) == 30*10**18  

In [123]:
u_c_instance.getOwner()

'0xe63419B7c0be62A6127923fb70C32A5ca1926a16'

In [124]:
u_c_instance.getStudent()

'0x6d031A36DD43f2fC2539037f0Bf8A0C818Ef9B59'

In [125]:
w3.parity.personal.listAccounts()

['0xe63419B7c0be62A6127923fb70C32A5ca1926a16']

In [126]:
# copy challenge metadata
challenge_path="/smartcode/challenges/badparity/publish/*"
attach_path="/smartcode/generated_config/attach"
challenge_name="badparity"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartcode/generated_config/attach/d7bdb839a8b3500ff9db3a536de3fdc1ef90a7c6/badparity
/smartcode/generated_config/attach/9f2874ebca0e202e6f32deec20e7c03b1b0b5d8c/badparity
/smartcode/generated_config/attach/561f30aa118eb27a1e717a80e011e3ce92ad1e39/badparity
/smartcode/generated_config/attach/3e93bc40c509e3b8f8eb4a0e02bfdfcb7dcb16cd/badparity
/smartcode/generated_config/attach/55941e806f8ff9a40db959e08f2fb9b466141667/badparity
/smartcode/generated_config/attach/0f60ea75ffa7ebdb5ac2924056d3dcad33504932/badparity
/smartcode/generated_config/attach/e2f6f489afefd074b4907977bd125f6bb20d209d/badparity
/smartcode/generated_config/attach/7f44a60a059be711cfa2f8b5809019813a0ce03d/badparity
/smartcode/generated_config/attach/e87f9b2bb30497579c98fb70a6f21275ad18a4b2/badparity
/smartcode/generated_config/attach/2613c370f5e2646d2fba8a7652dca4544ee3bfa5/badparity
/smartcode/generated_config/attach/beab9297f2b63e43541b918c75ead656d231b986/badparity
/smartcode/generated_config/attach/c43463f718678d53605

## Challenge: Daodown

In [127]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'aljosha.judmayer@univie.ac.at',
 'matr': '1',
 'account': '6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'filename': 'UTC--2020-11-17T17-45-22.602842072Z--6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'attach': '/tmp/attach/6d031a36dd43f2fc2539037f0bf8a0c818ef9b59.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x4e09544c87b640268c02286e161a8001e754471b6c1c240882df20717e9fb3fd',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x2e52F50386de9fDF7177cb7C886B525595AAb634',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x952943aab82306d562752dc63672eb3d52fa3908ede0c9cb6830a395570a905e',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0x96cdA09224CE798b9d39742801A34486E1599a3f',
 'badparity_lib': '0x5bE9a46AEDE5cdF200E7aCfBfd6FcA7cF336F9CE'}

In [128]:
!solc --overwrite -o "/smartcode/challenges/daodown/publish/" --abi --bin "/smartcode/challenges/daodown/publish/EDao.sol"

Compiler run successful. Artifact(s) can be found in directory /smartcode/challenges/daodown/publish/.


In [129]:
path="/smartcode/challenges/daodown/publish/EDao.sol"
challange_name="daodown"
seedvalue=30*10**18

forall_compile_and_deploy(users_list,path,challange_name,seedvalue)
users_list[0]

Started for 18 users:
..................------------------

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'aljosha.judmayer@univie.ac.at',
 'matr': '1',
 'account': '6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'filename': 'UTC--2020-11-17T17-45-22.602842072Z--6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'attach': '/tmp/attach/6d031a36dd43f2fc2539037f0bf8a0c818ef9b59.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x4e09544c87b640268c02286e161a8001e754471b6c1c240882df20717e9fb3fd',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x2e52F50386de9fDF7177cb7C886B525595AAb634',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x952943aab82306d562752dc63672eb3d52fa3908ede0c9cb6830a395570a905e',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0x96cdA09224CE798b9d39742801A34486E1599a3f',
 'badparity_lib': '0x5bE9a46AEDE5cdF200E7aCfBfd6FcA7cF336F9CE',
 'daodown_tx': '0xef78616d7e8f504b343994a88cb004a9b16ffcaed5990c8f2d659fb3f9403207',
 'da

In [130]:
w3.eth.blockNumber

164

In [131]:
w3.eth.getBlock(w3.eth.blockNumber)["gasUsed"]

8336313

In [132]:
w3.eth.getBlock(w3.eth.blockNumber-3)["gasUsed"]

0

In [133]:
path="/smartcode/challenges/daodown/publish/EDao.sol"
challange_name="daodown"
forall_check_contract(users_list,path,challange_name)
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a < b))
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a == 10**18))
users_list[0]

Started for 18 users:
..................

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'aljosha.judmayer@univie.ac.at',
 'matr': '1',
 'account': '6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'filename': 'UTC--2020-11-17T17-45-22.602842072Z--6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'attach': '/tmp/attach/6d031a36dd43f2fc2539037f0bf8a0c818ef9b59.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x4e09544c87b640268c02286e161a8001e754471b6c1c240882df20717e9fb3fd',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x2e52F50386de9fDF7177cb7C886B525595AAb634',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x952943aab82306d562752dc63672eb3d52fa3908ede0c9cb6830a395570a905e',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0x96cdA09224CE798b9d39742801A34486E1599a3f',
 'badparity_lib': '0x5bE9a46AEDE5cdF200E7aCfBfd6FcA7cF336F9CE',
 'daodown_tx': '0xef78616d7e8f504b343994a88cb004a9b16ffcaed5990c8f2d659fb3f9403207',
 'da

In [134]:
# copy challenge metadata
challenge_path="/smartcode/challenges/daodown/publish/*"
attach_path="/smartcode/generated_config/attach"
challenge_name="daodown"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartcode/generated_config/attach/d7bdb839a8b3500ff9db3a536de3fdc1ef90a7c6/daodown
/smartcode/generated_config/attach/9f2874ebca0e202e6f32deec20e7c03b1b0b5d8c/daodown
/smartcode/generated_config/attach/561f30aa118eb27a1e717a80e011e3ce92ad1e39/daodown
/smartcode/generated_config/attach/3e93bc40c509e3b8f8eb4a0e02bfdfcb7dcb16cd/daodown
/smartcode/generated_config/attach/55941e806f8ff9a40db959e08f2fb9b466141667/daodown
/smartcode/generated_config/attach/0f60ea75ffa7ebdb5ac2924056d3dcad33504932/daodown
/smartcode/generated_config/attach/e2f6f489afefd074b4907977bd125f6bb20d209d/daodown
/smartcode/generated_config/attach/7f44a60a059be711cfa2f8b5809019813a0ce03d/daodown
/smartcode/generated_config/attach/e87f9b2bb30497579c98fb70a6f21275ad18a4b2/daodown
/smartcode/generated_config/attach/2613c370f5e2646d2fba8a7652dca4544ee3bfa5/daodown
/smartcode/generated_config/attach/beab9297f2b63e43541b918c75ead656d231b986/daodown
/smartcode/generated_config/attach/c43463f718678d53605bac149bd17038ed9a5e5e/

## Challenge: Faildice

In [135]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'aljosha.judmayer@univie.ac.at',
 'matr': '1',
 'account': '6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'filename': 'UTC--2020-11-17T17-45-22.602842072Z--6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'attach': '/tmp/attach/6d031a36dd43f2fc2539037f0bf8a0c818ef9b59.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x4e09544c87b640268c02286e161a8001e754471b6c1c240882df20717e9fb3fd',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x2e52F50386de9fDF7177cb7C886B525595AAb634',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x952943aab82306d562752dc63672eb3d52fa3908ede0c9cb6830a395570a905e',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0x96cdA09224CE798b9d39742801A34486E1599a3f',
 'badparity_lib': '0x5bE9a46AEDE5cdF200E7aCfBfd6FcA7cF336F9CE',
 'daodown_tx': '0xef78616d7e8f504b343994a88cb004a9b16ffcaed5990c8f2d659fb3f9403207',
 'da

In [136]:
!ls -la /usr/local/bin/solc*

-rwxr-xr-x 1 root root  6284040 Nov 13 16:11 /usr/local/bin/solc
-rwxr-xr-x 1 root root  6284040 Feb 12  2019 /usr/local/bin/solc_5.4
-rwxr-xr-x 1 root root 11192752 Oct 19 16:17 /usr/local/bin/solc_7.4


In [137]:
!/usr/local/bin/solc --version

solc, the solidity compiler commandline interface
Version: 0.5.4+commit.9549d8ff.Linux.g++


In [138]:
!solc --overwrite -o "/smartcode/challenges/faildice/publish/" --abi --bin "/smartcode/challenges/faildice/publish/SatoshiFailDice.sol"

Compiler run successful. Artifact(s) can be found in directory /smartcode/challenges/faildice/publish/.


In [139]:
path="/smartcode/challenges/faildice/publish/SatoshiFailDice.sol"
challange_name="faildice"
seedvalue=30*10**18

forall_compile_and_deploy(users_list,path,challange_name,seedvalue,argument2=True)
users_list[0]

Started for 18 users:
~~~~~~~~~~~~~~~~~~------------------

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'aljosha.judmayer@univie.ac.at',
 'matr': '1',
 'account': '6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'filename': 'UTC--2020-11-17T17-45-22.602842072Z--6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'attach': '/tmp/attach/6d031a36dd43f2fc2539037f0bf8a0c818ef9b59.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x4e09544c87b640268c02286e161a8001e754471b6c1c240882df20717e9fb3fd',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x2e52F50386de9fDF7177cb7C886B525595AAb634',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x952943aab82306d562752dc63672eb3d52fa3908ede0c9cb6830a395570a905e',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0x96cdA09224CE798b9d39742801A34486E1599a3f',
 'badparity_lib': '0x5bE9a46AEDE5cdF200E7aCfBfd6FcA7cF336F9CE',
 'daodown_tx': '0xef78616d7e8f504b343994a88cb004a9b16ffcaed5990c8f2d659fb3f9403207',
 'da

In [140]:
path="/smartcode/challenges/faildice/publish/SatoshiFailDice.sol"
challange_name="faildice"
forall_check_contract(users_list,path,challange_name)
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a < b))
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a == 10**18))
users_list[0]

Started for 18 users:
..................

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'aljosha.judmayer@univie.ac.at',
 'matr': '1',
 'account': '6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'filename': 'UTC--2020-11-17T17-45-22.602842072Z--6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'attach': '/tmp/attach/6d031a36dd43f2fc2539037f0bf8a0c818ef9b59.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x4e09544c87b640268c02286e161a8001e754471b6c1c240882df20717e9fb3fd',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x2e52F50386de9fDF7177cb7C886B525595AAb634',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x952943aab82306d562752dc63672eb3d52fa3908ede0c9cb6830a395570a905e',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0x96cdA09224CE798b9d39742801A34486E1599a3f',
 'badparity_lib': '0x5bE9a46AEDE5cdF200E7aCfBfd6FcA7cF336F9CE',
 'daodown_tx': '0xef78616d7e8f504b343994a88cb004a9b16ffcaed5990c8f2d659fb3f9403207',
 'da

In [141]:
# copy challenge metadata
challenge_path="/smartcode/challenges/faildice/publish/*"
attach_path="/smartcode/generated_config/attach"
challenge_name="faildice"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartcode/generated_config/attach/d7bdb839a8b3500ff9db3a536de3fdc1ef90a7c6/faildice
/smartcode/generated_config/attach/9f2874ebca0e202e6f32deec20e7c03b1b0b5d8c/faildice
/smartcode/generated_config/attach/561f30aa118eb27a1e717a80e011e3ce92ad1e39/faildice
/smartcode/generated_config/attach/3e93bc40c509e3b8f8eb4a0e02bfdfcb7dcb16cd/faildice
/smartcode/generated_config/attach/55941e806f8ff9a40db959e08f2fb9b466141667/faildice
/smartcode/generated_config/attach/0f60ea75ffa7ebdb5ac2924056d3dcad33504932/faildice
/smartcode/generated_config/attach/e2f6f489afefd074b4907977bd125f6bb20d209d/faildice
/smartcode/generated_config/attach/7f44a60a059be711cfa2f8b5809019813a0ce03d/faildice
/smartcode/generated_config/attach/e87f9b2bb30497579c98fb70a6f21275ad18a4b2/faildice
/smartcode/generated_config/attach/2613c370f5e2646d2fba8a7652dca4544ee3bfa5/faildice
/smartcode/generated_config/attach/beab9297f2b63e43541b918c75ead656d231b986/faildice
/smartcode/generated_config/attach/c43463f718678d53605bac149bd170

## Write Students

In [142]:
import json

def dump_users(path,users):
    with open(path,'w') as f_output:
        json.dump(users,f_output)

In [143]:
out_file_json = "/smartcode/generated_config/students.json"
dump_users(out_file_json,users_list)
len(users_list)

18

In [144]:
import csv

def write_students(path,
                   out_fieldnames=['firstname', 'lastname','matr','email','cc','attach','contract','account'],
                   cc_mail="ajudmayer@sba-research.org"):
    with open(path,'w',encoding='utf-8') as f_output:
    #with open(path,'w',encoding='latin-1') as f_output:
        writer = csv.DictWriter(f_output, 
                                fieldnames=out_fieldnames, 
                                delimiter=',',
                                quotechar='\"',
                                extrasaction='ignore')
        writer.writeheader()
        for i,row in enumerate(users_list):
            row["cc"]=cc_mail
            #print(row)
            writer.writerow(row)
    return i+1

In [145]:
out_file = "/smartcode/generated_config/students.csv"
out_fieldnames = ['matr','email','cc','attach','account']
#out_fieldnames = ['firstname', 'lastname','matr','email','cc','attach','contract','account']
#out_fieldnames = ['firstname', 
#                  'lastname',
#                  'matr',
#                  'email',
#                  'cc',
#                  'attach',
#                  'account',
#                  'notawallet_addr',
#                 ]
cc_mail = "ajudmayer@sba-research.org"
write_students(out_file,out_fieldnames,cc_mail)

18

In [146]:
!head -n2 $out_file

In [147]:
!wc -l $out_file

19 /smartcode/generated_config/students.csv


## Prepare mail

In [148]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'aljosha.judmayer@univie.ac.at',
 'matr': '1',
 'account': '6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'filename': 'UTC--2020-11-17T17-45-22.602842072Z--6d031a36dd43f2fc2539037f0bf8a0c818ef9b59',
 'attach': '/tmp/attach/6d031a36dd43f2fc2539037f0bf8a0c818ef9b59.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x4e09544c87b640268c02286e161a8001e754471b6c1c240882df20717e9fb3fd',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x2e52F50386de9fDF7177cb7C886B525595AAb634',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x952943aab82306d562752dc63672eb3d52fa3908ede0c9cb6830a395570a905e',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0x96cdA09224CE798b9d39742801A34486E1599a3f',
 'badparity_lib': '0x5bE9a46AEDE5cdF200E7aCfBfd6FcA7cF336F9CE',
 'daodown_tx': '0xef78616d7e8f504b343994a88cb004a9b16ffcaed5990c8f2d659fb3f9403207',
 'da

In [149]:
import os

def dump_addresses(users,attach_path="/smartcode/generated_config/attach"):
    for u in users:
        arg_dest = attach_path + "/" + u["account"] + "/" + "addresses"
        ch_string = "notawallet_addr = " + u["notawallet_addr"]
        !echo $ch_string >> $arg_dest
        
        ch_string = "badparity_addr = " + u["badparity_addr"]
        !echo $ch_string >> $arg_dest    
        
        ch_string = "daodown_addr = " + u["daodown_addr"]
        !echo $ch_string >> $arg_dest
        
        ch_string = "faildice_addr = " + u["faildice_addr"]
        !echo $ch_string >> $arg_dest
        
        print(".",end="")

In [150]:
# write all addresses to folders 
dump_addresses(users_list,attach_path="/smartcode/generated_config/attach")

..................

In [151]:
!cp /smartcode/passwordfile /smartcode/general_info/

In [152]:
# copy general information i.e., datadir to individual folders
source_path="/smartcode/general_info/*"
attach_path="/smartcode/generated_config/attach"
challenge_name=""
copy_challenge_metadata(users_list,source_path,challenge_name,attach_path)

/smartcode/generated_config/attach/d7bdb839a8b3500ff9db3a536de3fdc1ef90a7c6/
/smartcode/generated_config/attach/9f2874ebca0e202e6f32deec20e7c03b1b0b5d8c/
/smartcode/generated_config/attach/561f30aa118eb27a1e717a80e011e3ce92ad1e39/
/smartcode/generated_config/attach/3e93bc40c509e3b8f8eb4a0e02bfdfcb7dcb16cd/
/smartcode/generated_config/attach/55941e806f8ff9a40db959e08f2fb9b466141667/
/smartcode/generated_config/attach/0f60ea75ffa7ebdb5ac2924056d3dcad33504932/
/smartcode/generated_config/attach/e2f6f489afefd074b4907977bd125f6bb20d209d/
/smartcode/generated_config/attach/7f44a60a059be711cfa2f8b5809019813a0ce03d/
/smartcode/generated_config/attach/e87f9b2bb30497579c98fb70a6f21275ad18a4b2/
/smartcode/generated_config/attach/2613c370f5e2646d2fba8a7652dca4544ee3bfa5/
/smartcode/generated_config/attach/beab9297f2b63e43541b918c75ead656d231b986/
/smartcode/generated_config/attach/c43463f718678d53605bac149bd17038ed9a5e5e/
/smartcode/generated_config/attach/44abc2d5e268f47d83749972041db7db845babcb/

In [156]:
# create archives
import os

def create_user_archives(users,attach_path="/smartcode/generated_config/attach",attach_tgz="/smartcode/generated_config/attach_tgz"):
    !mkdir -p $attach_tgz
    for d in os.listdir(attach_path):
        #print(d)
        if os.path.isdir(os.path.join(attach_path, d)):
            #print(d)
            arg_src_dir = attach_path + "/" + d 
            arg_dst_dir = attach_tgz + "/" + d + ".tar.gz"
            print(arg_dst_dir)
            #With verbose output:
            #!tar -cvzf $arg_dst_dir -C $arg_src_dir . 
            #Without verbose output
            !tar -czf $arg_dst_dir -C $arg_src_dir . 

In [157]:
attach_path="/smartcode/generated_config/attach"
attach_tgz="/smartcode/generated_config/attach_tgz"
create_user_archives(users_list,attach_path,attach_tgz)

/smartcode/generated_config/attach_tgz/d7bdb839a8b3500ff9db3a536de3fdc1ef90a7c6.tar.gz
/smartcode/generated_config/attach_tgz/9f2874ebca0e202e6f32deec20e7c03b1b0b5d8c.tar.gz
/smartcode/generated_config/attach_tgz/561f30aa118eb27a1e717a80e011e3ce92ad1e39.tar.gz
/smartcode/generated_config/attach_tgz/3e93bc40c509e3b8f8eb4a0e02bfdfcb7dcb16cd.tar.gz
/smartcode/generated_config/attach_tgz/55941e806f8ff9a40db959e08f2fb9b466141667.tar.gz
/smartcode/generated_config/attach_tgz/0f60ea75ffa7ebdb5ac2924056d3dcad33504932.tar.gz
/smartcode/generated_config/attach_tgz/e2f6f489afefd074b4907977bd125f6bb20d209d.tar.gz
/smartcode/generated_config/attach_tgz/7f44a60a059be711cfa2f8b5809019813a0ce03d.tar.gz
/smartcode/generated_config/attach_tgz/e87f9b2bb30497579c98fb70a6f21275ad18a4b2.tar.gz
/smartcode/generated_config/attach_tgz/2613c370f5e2646d2fba8a7652dca4544ee3bfa5.tar.gz
/smartcode/generated_config/attach_tgz/beab9297f2b63e43541b918c75ead656d231b986.tar.gz
/smartcode/generated_config/attach_tgz/c434